登入

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# 處理敏感資訊
from dotenv import load_dotenv
import os

load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")

options = Options()
# 無頭
# options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--lang=zh-TW")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
driver.get("https://member.tw.coupang.com/login/login.pang")
# 等待兩秒，確保頁面加載完成
time.sleep(2)

# 輸入帳號
email_input = driver.find_element(
    By.ID, "login-email-input"
)
email_input.send_keys(EMAIL)

# 輸入密碼
password_input = driver.find_element(
    By.ID, "login-password-input"
)
password_input.send_keys(PASSWORD)

# 點擊登入按鈕
login_button = driver.find_element(
    By.CLASS_NAME, "login__button--submit"
)
login_button.click()

# 等待跳轉
time.sleep(5)

# 可自行接著進入訂單頁、商品頁
print("✅ 登入流程已觸發")
# 結束時再關閉瀏覽器
# driver.quit()

✅ 登入流程已觸發


登入後查詢

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import quote
import time
from dotenv import load_dotenv
import os

# 敏感資訊
load_dotenv()
EMAIL = os.getenv("COUPANG_EMAIL")
PASSWORD = os.getenv("COUPANG_PASSWORD")

options = Options()
# 無頭模式
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--lang=zh-TW")

# 共用 driver（登入後重用）
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

# === 登入流程 ===
driver.get("https://member.tw.coupang.com/login/login.pang")
time.sleep(2)

email_input = driver.find_element(
    By.ID, "login-email-input"
)
email_input.send_keys(EMAIL)

password_input = driver.find_element(
    By.ID, "login-password-input"
)
password_input.send_keys(PASSWORD)

login_button = driver.find_element(
    By.CLASS_NAME, "login__button--submit"
)
login_button.click()
time.sleep(5)

print("✅ 已登入 Coupang")


def get_coupang_search_results(driver, search_keyword: str, advanced_keywords: list[str]):
    encoded_keyword = quote(search_keyword)
    url = f"https://www.tw.coupang.com/search?q={encoded_keyword}&channel=user"
    driver.get(url)
    time.sleep(5)

    html = driver.page_source

    # 若需重複操作瀏覽器可先保留不關閉
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")
    product_cards = soup.select(
        "div.SearchResult_searchResultProduct___h6E9"
    )

    results = []
    matched_results = []

    for card in product_cards:
        try:
            full_text = card.get_text(separator=" ", strip=True)

            title_tag = card.select_one("div.Product_title__8K0xk")
            title = title_tag.get_text(strip=True) if title_tag else "N/A"

            price_tag = card.select_one("span.Product_salePricePrice__2FbsL span")
            price = price_tag.get_text(strip=True) if price_tag else "N/A"

            unit_price_tag = card.select_one("div.Product_unitPrice__QQPdR")
            unit_price = unit_price_tag.get_text(strip=True) if unit_price_tag else "N/A"

            product = {
                "title": title,
                "full_text": full_text,
                "price": price,
                "unit_price": unit_price
            }

            results.append(product)

            if all(kw in full_text for kw in advanced_keywords):
                matched_results.append(product)

        except Exception as e:
            print("解析錯誤：", e)
            continue

    return results, matched_results


if __name__ == "__main__":
    # 關鍵字搜尋
    search_keyword = "味丹 氣泡水"
    # 針對結果進階篩選
    advanced_keywords = ["百香果風味"]

    all_products, filtered_products = get_coupang_search_results(
        driver,
        search_keyword, 
        advanced_keywords
    )

    driver.quit()
    # 輸出關鍵字
    print(f"搜尋關鍵字：{search_keyword}")

    # 完整查詢結果
    for idx, product in enumerate(all_products, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)

    # 進階查詢
    print(f"\n進階條件符合項目（包含：{'、'.join(advanced_keywords)}）：")
    for idx, product in enumerate(filtered_products, 1):
        print(f"{idx}. 標題: {product['title']}")
        print(f"   價格: {product['price']}")
        print(f"   每單位: {product['unit_price']}")
        print(f"   完整內容: {product['full_text']}")
        print("-" * 60)

✅ 已登入 Coupang
搜尋關鍵字：味丹 氣泡水
1. 標題: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 24瓶
   價格: $387
   每單位: ($2.88/100ml)
   完整內容: 味丹 多喝水 MORE氣泡水 檸檬風味, 560ml, 24瓶 特價 63折 $624 $387 ($2.88/100ml) 明天 5/13 (二) 預計送達 免運 ( 911 )
------------------------------------------------------------
2. 標題: 味丹 MORE氣泡水 蜜桃風味, 560ml, 24瓶
   價格: $387
   每單位: ($2.88/100ml)
   完整內容: 味丹 MORE氣泡水 蜜桃風味, 560ml, 24瓶 特價 63折 $624 $387 ($2.88/100ml) 明天 5/13 (二) 預計送達 免運 ( 721 )
------------------------------------------------------------
3. 標題: 味丹 多喝水 MORE 原味氣泡水, 560ml, 24瓶
   價格: $387
   每單位: ($2.88/100ml)
   完整內容: 味丹 多喝水 MORE 原味氣泡水, 560ml, 24瓶 特價 63折 $624 $387 ($2.88/100ml) 明天 5/13 (二) 預計送達 免運 ( 326 )
------------------------------------------------------------
4. 標題: Tamsaa 無標籤氣泡水 原味, 500ml, 40瓶
   價格: $508
   每單位: ($2.54/100ml)
   完整內容: Tamsaa 無標籤氣泡水 原味, 500ml, 40瓶 $508 ($2.54/100ml) 明天 5/13 (二) 預計送達 免運 ( 7 )
------------------------------------------------------------
5. 標題: 多喝水 MORE+維他命氣泡水 百香果風味, 560ml, 24瓶
   價格: $489
   每單位: ($3.64/1